<a href="https://colab.research.google.com/github/ana-isabellagf/gee-landcover-analysis/blob/main/notebooks/extract_lulc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Landuse/Landcover (LULC) Time Series**
## **Extraction and Visualization**

Medium article: [LULC Time Series](https://medium.com/@aisabellaguimaraesf/série-temporal-do-uso-e-cobertura-do-solo-lulc-extração-e-visualização-52c59c5e0998)

Author: Ana Isabella Guimarães Ferreira (aisabellaguimaraesf@gmail.com)

LinkedIn: www.linkedin.com/in/ana-isabella-g-ferreira


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q rasterio

In [3]:
# Requirements
import time
import os
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio import features
from rasterio.mask import mask
import geemap
from shapely.geometry import shape
import folium
from folium.plugins import TimestampedGeoJson
from shapely.geometry import mapping
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-aisabellaguimaraesf')

In [ ]:
# CRS CODE - Brazil Data Cube
# https://brazil-data-cube.github.io/products/specifications/bdc-grid.html
crs_code = '+proj=aea +lat_0=-12 +lon_0=-54 +lat_1=-2 +lat_2=-22 +x_0=5000000 +y_0=10000000 +ellps=GRS80 +units=m +no_defs'

In [ ]:
def extract_lulc_images(start_year, end_year, geom, aoi, crs_code):
    """
    Extracts LULC images from Earth Engine for a given area of interest and returns a dictionary of GeoDataFrames by year.

    Args:
        start_year (int): Start year.
        end_year (int): End year.
        geom (ee.Geometry): Geometry of the area of interest.
        aoi (gpd.GeoDataFrame): GeoDataFrame containing the AOI boundaries.
        crs_code (str): CRS code for the resulting GeoDataFrames.

    Returns:
        dict: Dictionary containing LULC GeoDataFrames per year.
    """
    lulc_by_year = {}

    # Get start time
    start_time = time.time()

    for year in range(start_year, end_year + 1):
        # Access MapBiomas asset
        dataset = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1').select(f'classification_{year}').clip(geom)
        output_filename = f'temp_mapbiomas_{year}.tif'
        geemap.ee_export_image(dataset, output_filename, scale=30, crs="EPSG:4326", region=geom)

        # Open image locally using rasterio
        with rasterio.open(output_filename) as src:
            # Mask the raster using the AOI geometry
            out_image, out_transform = mask(src, aoi.geometry, crop=True)
            out_meta = src.meta.copy()

        # Extract features from raster
        shapes = features.shapes(out_image[0], transform=out_transform)

        # Lists to store LULC class values and geometries
        class_values = []
        geometries = []

        # Fill the lists with geometry and class values
        for shapedict, value in shapes:
            class_values.append(value)
            geometries.append(shape(shapedict))

        # Create a GeoDataFrame with class values and geometry
        lulc_gdf = gpd.GeoDataFrame({f'lulc_{year}': class_values, 'geometry': geometries}, crs=crs_code)

        # Store the GeoDataFrame in the dictionary
        lulc_by_year[year] = lulc_gdf

        # Remove temporary TIFF file
        os.remove(output_filename)

    # Get end time
    end_time = time.time()

    # Calculate and print execution time
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time:.2f} seconds")

    return lulc_by_year

# Area of interest
aoi = gpd.read_file('/content/drive/MyDrive/GitHub/lulc_dynamic/aoi_lulc_dynamic.zip')

# Geometry for image extraction
bounds = aoi.geometry.total_bounds
geom = ee.Geometry.Rectangle(bounds.tolist())

# Run the function
lulc_by_year = extract_lulc_images(1985, 2023, geom, aoi, crs_code)

In [ ]:
# Extract latitude and longitude
centroid = aoi.centroid.geometry.iloc[0]
lon = centroid.x
lat = centroid.y

# MapBiomas legend
# https://brasil.mapbiomas.org/wp-content/uploads/sites/4/2024/08/Codigos-da-legenda-colecao-9.zip
legend = pd.read_excel('/content/drive/MyDrive/GitHub/lulc_dynamic/Codigos-da-legenda-colecao-9.xlsx')
legend.head()

# Create a dictionary mapping class IDs to colors
class_color_map = dict(zip(legend['Class_ID'], legend['Color']))

In [ ]:
# Function to convert a GeoDataFrame into GeoJSON
def gdf_to_geojson(gdf, properties, date, class_color_map):
    features = []
    for _, row in gdf.iterrows():
        feature = {
            'type': 'Feature',
            'geometry': mapping(row.geometry),
            'properties': {prop: row[prop] for prop in properties}
        }
        # Add color property based on LULC class
        lulc_value = row[properties[0]]
        feature['properties']['color'] = class_color_map.get(lulc_value, "#000000")  # Black as fallback
        # Add timestamp property
        feature['properties']['time'] = date
        # Add LULC class name
        label = legend.loc[legend["Class_ID"] == lulc_value, "Description"].values
        if len(label) > 0:
            feature['properties']['class_name'] = label[0]
        else:
            feature['properties']['class_name'] = f'Class {lulc_value}'  # Default label if class is not in 'legend'
        features.append(feature)
    return {'type': 'FeatureCollection', 'features': features}

# Style function for the GeoJSON
def style_function(feature):
    return {
        'fillColor': feature['properties']['color'],
        'color': None,
        'weight': 0,
        'fillOpacity': 1,
    }

In [ ]:
# List to store the GeoJSONs
geojson_list = []
unique_classes = set()  # Set to store unique LULC classes

# Iterate over the years and convert GeoDataFrames to GeoJSON
for year_plot in range(2012, 2023):
    gdf = lulc_by_year[year_plot]
    # Convert float values to integers in the lulc_year column
    gdf[f'lulc_{year_plot}'] = gdf[f'lulc_{year_plot}'].astype(int)
    # Add unique classes to the set
    unique_classes.update(gdf[f'lulc_{year_plot}'].unique())
    # Convert GeoDataFrame to GeoJSON
    geojson = gdf_to_geojson(gdf, properties=[f'lulc_{year_plot}'], date=f'{year_plot}-01-01', class_color_map=class_color_map)
    geojson_list.append(geojson)

# Create a list of GeoJSONs with applied styling
styled_features = []
for geojson in geojson_list:
    for feature in geojson['features']:
        feature['properties']['style'] = style_function(feature)
        styled_features.append(feature)

In [ ]:
# Define the center coordinates of your area of interest
map_center = (lat, lon)
initial_zoom = 12.5

# Create base map
m = folium.Map(location=map_center, zoom_start=initial_zoom)

# Create GeoJsonTooltip
tooltip = folium.GeoJsonTooltip(
    fields=['class_name'],
    aliases=['Class: '],
    localize=True
)

# Create the temporal layer for the map
timestamped_geojson = TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': styled_features
    },
    transition_time=200,       # transition time between years
    period='P1Y',              # period of 1 year
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY',
    time_slider_drag_update=True
)

# Add the timestamped layer to the map
timestamped_geojson.add_to(m)

# Add GeoJson with tooltips to the map
folium.GeoJson(
    timestamped_geojson.data,
    style_function=style_function,
    tooltip=tooltip
).add_to(m)

# Save the map as an HTML file
m.save('LULC_history.html')